In [ ]:
import rvt
import rvt.vis  # for calculating visualizations
import rvt.default as default  # for loading/saving rasters
import rvt.blend
import rvt.VAT_combined
import os
import arcpy
from arcpy import env
from arcpy.sa import *
from pathlib import Path
import numpy as np

'''
This package customizes the base RVT Python functionalities for single-step visualization
processing. The functions establish necessary file paths, detect and process input DEMs 
for visualization, and finally creates the visualizations. These functions created all the 2.5D models
used for feature classification for the SCIP 2023 and 2024 field seasons.

Required packages: RVT, os, arcpy, arcpy.sa, pathlib, and numpy.
'''

def setOutputs():
    '''
    This function sets the output folder for export of visualizations.
    It checks whether a valid outputs folder exists in the current working directory. It does not require any arguments.
    If a valid folder exists, it returns the folder path.
    If a valid folder does not exist, a folder named "Outputs" is created and returned.
    
    It requires the os package
    
    Usage:
    >>setOutputs()
    '''
    folders = os.listdir()
    if "Outputs" in folders:       #Sorting through folders in directory to find Output(s) folder
        for folder in folders:     
            if folder == "Outputs":
                outPath = folder
                return os.path.abspath(outPath)     #Returns valid output folder as outPath
            else:
                continue
    elif "Output" in folders:
        for folder in folders:
            if folder == "Output":
                outPath = folder
                return os.path.abspath(outPath)
            else:
                continue
    else:
        os.makedirs("Outputs")    #If no valid output(s) folder, one will be created...
        folders = os.listdir()
        for folder in folders:
            if folder == "Outputs":
                outPath = folder
                return os.path.abspath(outPath)    #...and returned as outPath
            else:
                continue
        print("No outputs folder found. A folder named 'Outputs' has been created.")
        
def findDEMfolder():
    '''
    This function automatically detects a DEM folder containing valid input rasters for package functionalities.
    If no folder named DEM or DEMs is found, a folder named DEM is automatically created in your current working directory.
    The function returns the DEM folder path.
    
    It does not require any arguments.
    It requires the os package.
    
    Usage:
    >>findDEMfolder()
    '''
    folders = os.listdir() #get list of folders in directory
    if "DEM" in folders:
        for folder in folders:   #Sorting through directory folders for valid DEM folder
            if folder == "DEM":
                DEMfolder = folder 
                return DEMfolder #returns DEMfolder
            else:
                continue
    elif "DEMs" in folders:
        for folder in folders:
            if folder == "DEMs":
                DEMfolder = folder
                return DEMfolder
            else:
                continue
    else:
        os.makedirs("DEM")       #If no valid DEM(s) folder, one will be created...
        folders = os.listdir()
        for folder in folders:
            if folder == "DEM":
                DEMfolder = folder
                return DEMfolder #...and returned as DEMfolder
            else:
                continue
        print("No DEM folder found. A folder named 'DEM' has been created. Please place all rasters needed for your analysis here.")

def findDEMs(DEMfolder):
    '''
    This function returns a list of DEM .tif files from an input DEM folder.
    It requires one argument: a folder path containing input raster DEMs.
    It uses the os package.
    
    Usage:
    >>findDEMs(filePath)
    '''
    demList = []
    demList = [file for file in os.listdir(DEMfolder) if file.endswith(".tif")] #creates list of DEM files in DEMfolder
    return demList #returns demList

def removeTempRVT():
    '''
    This function removes temporary files used for the RVT image generation process. 
    It is tailored to remove specific temporary files automatically within various other functions and is not to be used to remove files normally.
    
    It does not take any arguments.
    It requires the os and ArcPy packages.
    
    Usage:
    >>removeTempRVT()
    '''
    env.workspace = os.getcwd()
    env.overwriteOutput = True
    tempFiles = arcpy.ListRasters()
    for temp in tempFiles:
        if temp == "OpenPosTemp.tif":
            os.remove(temp)
        elif temp == "OpenNegTemp.tif":
            os.remove(temp)
        else:
            continue
            
def makeArrayDict(demList, DEMfolder, outPath):
    '''
    This function makes an array dictionary list containing array infor for each input raster in the DEM folder.
    Requires 3 arguments: a folder containing DEMs, a list of DEM files, and the outputs folder path. 
    It requires the os package.
    
    Usage:
    >>makeArrayDict(ItalyDEMsList, DEMfolder, Outputs)
    '''
    origCWD = os.getcwd()
    os.chdir(DEMfolder) #changes the working directory to DEMfolder
    demArrays = []
    for dem in demList:
        demArr = rvt.default.get_raster_arr(dem)
        demArr['FileName'] = dem
        demArrays.append(demArr)
        #Saves original DEM to outputs folder
        demFile = outPath+"\\"+dem
        rvt.default.save_raster(src_raster_path=dem, out_raster_path=demFile, out_raster_arr=demArr['array'], no_data=np.nan,
                                               e_type=6)
    os.chdir(origCWD)
    return demArrays

def slopeAspect(demArray, outPath, output_units="degree", ve_factor=1):
    '''
    This function returns a slope aspect array of an input array.
    It requires 2 arguments, a DEM array dictionary that follows Relief Visualization Toolbox standards and an Output file path
    It has two optional arguments: 1. output units, which defaults to "degree." Other options are "radian" and "percent"
                                                           2. vertical exaggeration factor, which defaults to 1.  
    This function requires the Relief Visualization Toolbox.
    Usage:
    >>slopeAspect(ApenninesDEM_Array, ApenninesOutputs)
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #Slope
    dictSlopeAspect = rvt.vis.slope_aspect(dem=dem_arr, resolution_x=dem_res_x, resolution_y=dem_res_y, 
                                                                         output_units=output_units, ve_factor=ve_factor, no_data=dem_no_data)
    slope_arr = dictSlopeAspect["slope"]
    slopeFile = outPath+"\\"+Path(dem_name).stem+"_Slope.tif"
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=slopeFile, out_raster_arr=slope_arr, no_data=np.nan,
                                             e_type=6)
    return slope_arr

def normalHS(demArray, outPath, sun_azimuth=315, sun_elevation=35, ve_factor=1):
    '''
    This function returns a normal hillshade array of an input array.
    It requires 5 arguments (2 required, 3 optional): 1. a DEM array dictionary that follows Relief Visualization Toolbox standards.
                                                                                         2. Outputs path
                                                                                         3. Sun Azimuth angle - clockwise from North - in degrees; default is 315
                                                                                         4. Sun Elevation angle - solar vertical above the horizon - in degrees; default is 35
                                                                                         5. Vertical Exaggeration factor; defaults to 1
                    
    This function requires the Relief Visualization Toolbox.
    Usage:
    >>normalHS(ApenninesDEM_Array, ApenninesOutputs)    
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #HS
    hillshade_arr = rvt.vis.hillshade(dem=dem_arr, resolution_x=dem_res_x, resolution_y=dem_res_y,
                                                              sun_azimuth=sun_azimuth, sun_elevation=sun_elevation, ve_factor=ve_factor)
    hsFile = outPath+"\\"+Path(dem_name).stem+f"_HS_{sun_azimuth}az_{sun_elevation}elev.tif"
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=hsFile, out_raster_arr=hillshade_arr, no_data=np.nan,
                                            e_type=6)
    return hillshade_arr

def mdHS(demArray, outPath, sun_azimuth=315, sun_elevation=35, nr_directions=16, ve_factor=1):
    '''
    This function returns a normal hillshade array of an input array.
    It has 6 arguments (2 required, 4 optional):   1. a DEM array dictionary that follows Relief Visualization Toolbox standards.
                                                                                  2. Outputs path
                                                                                  3. Sun Azimuth angle - clockwise from North - in degrees; default is 315
                                                                                  4. Sun Elevation angle - solar vertical above the horizon - in degrees; default is 35
                                                                                  5. Number of Solar Azimuth angles - clockwise from North; # of Directions = # of Bands; default is 16
                                                                                  6. Vertical Exaggeration factor; defaults to 1
    This function requires the Relief Visualization Toolbox.
    Usage:
    >>mdHS(ApenninesDEM_Array, ApenninesOutputs)        
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #Create MDHS
    multi_hillshade_arr = rvt.vis.multi_hillshade(dem=dem_arr, resolution_x=dem_res_x, resolution_y=dem_res_y,
                                                                                      nr_directions=nr_directions, sun_elevation=sun_elevation, ve_factor=ve_factor,
                                                                                      no_data=dem_no_data)
    mdhsFile = outPath+"\\"+Path(dem_name).stem+f"_mdHS_{sun_azimuth}az_{sun_elevation}elev_dir{nr_directions}.tif"
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=mdhsFile, out_raster_arr=multi_hillshade_arr,
                                            no_data=np.nan, e_type=6)
    return multi_hillshade_arr

def slrm(demArray, outPath, radius_cell=16, ve_factor=1):
    '''
    This function returns a Simple Local Relief Model array of an input array.
    It takes arguments (2 required, 2 optional): 1. a DEM array dictionary that follows Relief Visualization Toolbox standards.
                                                                                2. Outputs path file
                                                                                3. Radius Cell (in pixels); default is 16
                                                                                4. Vertical Exaggeration Factor; default is 1
    This function requires the Relief Visualization Toolbox.
    Usage:
    >>slrm(ApenninesDEM_Array)    
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #SLRM
    slrm_arr = rvt.vis.slrm(dem=dem_arr, radius_cell=radius_cell, ve_factor=ve_factor, no_data=dem_no_data)
    slrmFile = outPath+"\\"+Path(dem_name).stem+f"_SLRM_rad{radius_cell}.tif"
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=slrmFile, out_raster_arr=slrm_arr,
                                                no_data=np.nan, e_type=6)
    return slrm_arr

def opennessIndex(demArray, outPath, svf_n_dir=16, svf_r_max=10, svf_noise=0,asvf_level=1,asvf_dir=315, ve_factor=1):
    '''
    This function returns an Openness Index raster (.tif) file from an input array.
    It requires 7 arguments:   1. DEM Array
                                                 2. Number of Sky View Factor Directions; default = 16
                                                 3. Max Search Radius in Pixels for SVF; default is 10
                                                 4. Level of Noise Removal for SVF; default is 0; (0 = none, 1 = low, 2 = medium, 3 = high)
                                                 5. Level of Anisotropy; default = 1; (1 = low, 2 = high)
                                                 6. Direction of Anisotropy in Degrees; default = 315
                                                 7. Vertical Exaggeration Factor; default is 1
    This function requires the ArcPy Spatial Analysis module and the Relief Visualization Toolbox.
    Usage:
    >>opennessIndex(ApenninesDEM_Array, outputsFolder)
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #Create Sky View Factor Dict without Computing SVF/ASVF, just Openness
    dict_PosOpen = rvt.vis.sky_view_factor(dem=dem_arr, resolution=dem_res_x, compute_svf=False, compute_asvf=False, compute_opns=True,
                                           svf_n_dir=svf_n_dir, svf_r_max=svf_r_max, svf_noise=svf_noise,
                                           asvf_level=asvf_level, asvf_dir=asvf_dir,
                                           no_data=dem_no_data)
    posOpenArr = dict_PosOpen["opns"]  # positive openness
    #Negative Openness
    dem_arr_neg_opns = dem_arr * -1  #Multiple DEM array by -1 to compute Negative Openness
    dict_NegOpen = rvt.vis.sky_view_factor(dem=dem_arr_neg_opns, resolution=dem_res_x, compute_svf=False, compute_asvf=False, compute_opns=True,
                                           svf_n_dir=svf_n_dir, svf_r_max=svf_r_max, svf_noise=svf_noise,
                                       no_data=dem_no_data)
    negOpenArr = dict_NegOpen["opns"]  
    #Create temporary raster outputs in current working directory
    posRas = "OpenPosTemp.tif"
    negRas = "OpenNegTemp.tif"
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=posRas, out_raster_arr=posOpenArr,
                                        no_data=np.nan, e_type=6)
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=negRas, out_raster_arr=negOpenArr,
                                        no_data=np.nan, e_type=6)
    #Raster Calculator from arcpy.sa math methods
    oiFile = outPath+"\\"+Path(dem_name).stem+"_OpennessIndex.tif"
    OpennessIndex = RasterCalculator([posRas, negRas], ["x", "y"], #Openness Index formula
                                    "(x-y)/2", "UnionOf")
    OpennessIndex.save(oiFile)
    removeTempRVT()
    return oiFile

def LocalDom(demArray, outPath, min_rad = 10, max_rad = 16, rad_inc = 1, angular_res = 15, observer_height = 1.7, ve_factor=1):
    '''
    This function returns a Local Dominance array of an input array.
    It requires 2 arguments:  1. a DEM array dictionary that follows Relief Visualization Toolbox standards.
                                                2. Outputs folder path
    It takes 6 optional arguments:   1. Minimum Radial Distance; default = 10
                                                           2. Maximum Radial Distance; default = 16
                                                           3. Radial Distance Steps in Pixels; default = 1
                                                           4. Angular Step for Determination of Number of Angular Directions; default = 15
                                                           5. Observer Height (in meters); default = 1.7
                                                           6. Vertical Exaggeration factor; default is 1

    This function requires the Relief Visualization Toolbox.
    
    Usage:
    >>LocalDom(ApenninesDEM_Array, outputFolder)        
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #Local Dominance
    local_dom_arr = rvt.vis.local_dominance(dem=dem_arr, min_rad=min_rad, max_rad=max_rad, rad_inc=rad_inc, angular_res=angular_res,
                                            observer_height=observer_height, ve_factor=1,
                                            no_data=dem_no_data)
    LocalDomFile = outPath+"\\"+Path(dem_name).stem+"_LocalDominance.tif"
    rvt.default.save_raster(src_raster_path=dem_name, out_raster_path=LocalDomFile, out_raster_arr=local_dom_arr,
                                            no_data=np.nan, e_type=6)
    return local_dom_arr

def combinedVAT(DEMfolder, outPath, general_opacity=50):
    '''
    This function creates a default VAT blend. It reads an input folder for DEMs and saves an output visual to the outpath.
    It requires 2 arguments: a DEM folder and an output folder.
    It has one optional argument: the general opacity of the blend. This defaults to 50%
    
    This package requires a manual installation of rvt. Copy the rvt folder from the rvt_py-master into your current directory.
    Copy the "settings" folder and the VAT_combined.py file from the rvt_py-master folder and paste into the base rvt folder

    
    Usage:
    >>combinedVAT(ApennineDEMs, ApennineOutputs)
    >>combinedVAT(ApennineDEMs, ApennineOutputs, 75)
    '''
    vat_combination_json_path = os.getcwd()+"/rvt/settings/blender_VAT.json"
    terrains_sett_json_path = os.getcwd()+"/rvt/settings/default_terrains_settings.json"
    #Combined VAT
    rvt.VAT_combined.combined_VAT(input_dir_path=DEMfolder,output_dir_path=outPath, general_opacity=general_opacity,
                                                                    vat_combination_json_path=vat_combination_json_path,terrains_sett_json_path=terrains_sett_json_path)
    
def gradReliefBase(demArray, outPath):
    '''
    This function creates the base image blend used for the Gradient Relief Model visualization. 
    It uses the Relief Visualization Toolbox blend tools from rvt.blend and the os package.
    
    It requires two arguments: a DEM array and an output folder path
    
    Usage:
    >>gradReliefBase(ApennineDEMs, ApenninesOutputs)
    '''
    #DEM Info
    dem_name = demArray["FileName"]
    dem_arr = demArray["array"]
    dem_res = demArray["resolution"]
    dem_no_data = demArray["no_data"]
    dem_res_x = dem_res[0]
    dem_res_y = dem_res[1]
    #Blend
    layers = rvt.blend.BlenderCombination()
    input_dem_path = os. path. abspath(dem_name)
    layers.add_dem_path(dem_path=input_dem_path)
    gradReliefFile = outPath+"\\"+Path(dem_name).stem+"_GradientReliefBase.tif"
    input_dem_arr = dem_arr
    layers.add_dem_arr(input_dem_arr, dem_res_x)
    # layers;vis_method;norm;min;max;blending_mode;opacity
    #Layer 1 blend - Sky View Factor
    layers.create_layer(vis_method="Sky-View Factor", normalization="value", minimum=0.5, maximum=1,
                                      blend_mode="Multiply", opacity=25)  
    #Layer 2 blend - Openness Positive
    layers.create_layer(vis_method="Openness - Positive", normalization="value", minimum=68.0, maximum=93.0,
                                        blend_mode="Overlay", opacity=50)
    #Layer 3 blend - Slope Gradient
    layers.create_layer(vis_method="Slope gradient", normalization="value", minimum=0, maximum=50,
                                        blend_mode="Luminosity", opacity=50)
    #Layer 4 blend - Multi-directional hillshade
    layers.create_layer(vis_method="Multiple directions hillshade", normalization="value", minimum=0, maximum=1.5,
                                        blend_mode="Normal", opacity=100)
    layers.render_all_images(save_render_path=gradReliefFile)
    
def rvt_viz(demArrays, DEMfolder, outPath):
    '''
    This function creates Relief Visualization Toolbox images for each raster DEM in a folder.
    It uses custom functions created in the ManquenFunctionsRVT module to compute 6 visualizations.
    Visualizations: 1. Slope
                               2. Hillshade
                               3. Multi-directional hillshade
                               4. Simple Local Relief Model
                               5. Openness Index
                               6. Local Dominance
                               
    It requires 3 arguments: 1. a list of DEM arrays
                                               2. DEM folder path
                                               3. Outputs folder path
                                               
    This function requires ArcPy and os
    
    Usage:
    >>rvt_viz(ApenninesArrays, ApenninesDEM, ApenninesOutputs)
    '''
    origCWD = os.getcwd()
    os.chdir(DEMfolder) #changes the working directory to DEMfolder
    for demArray in demArrays:
        #Get Array Info
        dem_name = demArray["FileName"]
        print(f"Getting array info for {dem_name}...")
        print("")
        dem_arr = demArray["array"]
        dem_res = demArray["resolution"]
        dem_no_data = demArray["no_data"]
        dem_res_x = dem_res[0]
        dem_res_y = dem_res[1]
        ##RVT Basic Visualizations: Slope, Hillshade, Multidirectional Hillshade, SLRM, Pos/Neg Openness, Local Dom
        print(f"Visualizations in progress for {dem_name}...")
        slopeArr=slopeAspect(demArray, outPath)
        print("Slope model complete.")
        hsArr=normalHS(demArray, outPath)
        print("Hillshade model complete.")
        #mdHSarr=mdHS(demArray, outPath)
        #print("Multi-directional hillshade model complete.")
        slrmArr=slrm(demArray, outPath)
        print("Simple Local Relief Model complete.")
        oi=opennessIndex(demArray, outPath)
        print("Openness Index model complete.")
        LocalDomArr=LocalDom(demArray, outPath)
        print("Local Dominance model complete.")
        gradReliefBase(demArray, outPath)
        print("Gradient Relief Base model complete.")
        print("")
        #removeTempRVT()
        print("")
    #Create blends
    os.chdir(origCWD) #returns to original CWD
    env.workspace = origCWD
    combinedVAT(DEMfolder, outPath)
    print("Visualizations complete!")
    print(f"Find the outputs here: {outPath}")